<a href="https://colab.research.google.com/github/SushilKokare/AtiNeu_Project/blob/main/ensembleYT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# pip install Augmentor

In [4]:
# import Augmentor

In [5]:
# Here write augmentiation part

In [6]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten, Dropout, Input, Average, GlobalAveragePooling2D
from keras.models import Model
from keras.layers import BatchNormalization, MaxPooling2D

In [7]:
train_dir = '/content/drive/MyDrive/AtiNeu Project/Data/MRI/Brain_Data_Organised/Train'
test_dir = '/content/drive/MyDrive/AtiNeu Project/Data/MRI/Brain_Data_Organised/Test'

In [8]:
input_shape = (650,650)
batch_size = 10

In [9]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    seed = 123,
    image_size=input_shape,
    batch_size = batch_size,
    #color_mode = 'grayscale'
)

Found 2251 files belonging to 2 classes.


In [10]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    seed = 123,
    image_size = input_shape,
    batch_size = batch_size,
    #color_mode = 'grayscale'
)

Found 250 files belonging to 2 classes.


In [11]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [12]:
normalized_ds = train_ds.map(lambda x,y: (normalization_layer(x),y))
image_batch = labels_batch = next(iter(normalized_ds))
first_img = image_batch[0]
print(np.min(first_img),np.max(first_img))


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


0.0 1.0


In [13]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
test_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)

In [14]:
from tensorflow.keras.applications import VGG16,ResNet50
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications import InceptionV3

In [15]:
IV3_base_model = InceptionV3(
    input_shape = (650,650,3),
    weights = 'imagenet',
    include_top = False
)

87910968/87910968 [==============================] - 3s 0us/step


In [16]:
# Freeze  the first 10 layers
for layer in IV3_base_model.layers[:10]:
  layer.trainable = False

x = IV3_base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.4)(x)
predictions = Dense(2,activation='sigmoid')(x)

model1 = Model(inputs = IV3_base_model.inputs, outputs = predictions)

In [17]:
VGG16_base_model = VGG16(
    input_shape = (650,650,3),
    weights = 'imagenet',
    include_top = False
)

58889256/58889256 [==============================] - 2s 0us/step


In [18]:
for layer in VGG16_base_model.layers[:10]:
  layer.trainable = False

x = VGG16_base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.4)(x)
predictions = Dense(2,activation='sigmoid')(x)

model2 = Model(inputs = VGG16_base_model.inputs, outputs = predictions)

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [20]:
# checkpoint to save best model per epoch

model_filepath = '/content/drive/MyDrive/AtiNeu Project/model-{epoch:02d} - {val_accuracy: .4f}.hdf5'
checkpoint = ModelCheckpoint(
    filepath = model_filepath,
    monitor = 'val_accuracy',
    mode = 'max',
    save_best_only = True,
    verbose = 1
)

In [21]:
# compilations of models

In [22]:
model1.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

In [23]:
model2.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [ ]:
history1 = model1.fit(
    train_ds,
    validation_data = test_ds,
    epochs = 5,
    callbacks=[checkpoint]
)

Epoch 1/5


In [ ]:
#  import gc
#  gc.collect()

In [ ]:
# import matplotlib.pyplot as plt

# acc = history1.history['accuracy']
# val_acc = history1.history['val_accuracy']
# loss = hitory1.history['loss']
# val_loss = history1.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs,acc,'r',label='Train Acc')
# plt.plot(epochs,acc,'b',label='Val Acc')
# plt.title("Training & Validation Accuracy")
# plt.legend()
# plt.figure()

# plt.plot(epochs,loss,'r',label='Train Loss')
# plt.plot(epochs,acc,'b',label='Val Loss')
# plt.title("Training & Validation Loss")
# plt.legend()

# plt.show()

In [ ]:
# history2 = model2.fit(
#     train_ds,
#     validation_data = test_ds,
#     epochs = 5,
#     callbacks=[checkpoint]
# )

In [ ]:
# import matplotlib.pyplot as plt

# acc = history2.history['accuracy']
# val_acc = history2.history['val_accuracy']
# loss = hitory2.history['loss']
# val_loss = history2.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs,acc,'r',label='Train Acc')
# plt.plot(epochs,acc,'b',label='Val Acc')
# plt.title("Training & Validation Accuracy")
# plt.legend()
# plt.figure()

# plt.plot(epochs,loss,'r',label='Train Loss')
# plt.plot(epochs,acc,'b',label='Val Loss')
# plt.title("Training & Validation Loss")
# plt.legend()

# plt.show()

In [ ]:
# Combining Model

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input,Average

In [ ]:
# model_1 = load_model('/content/drive/MyDrive/model-01-0.9044.hdf5')
# model_1 = Model(Input=model_1.inputs,
#                 output=model_1.outputs,
#                 name='name_of_model_1')

In [ ]:
# model_2 = load_model('/content/drive/MyDrive/model-04-0.0156.hdf5')
# model_2 = Model(Input=model_2.inputs,
#                 output=model_2.outputs,
#                 name='name_of_model_2')

In [ ]:
# models = [model_1,model_2]
# model_input = Input(Shape=(224,224,3))
# model_outputs = [model(model_input) for model in models]
# ensemble_output = Average()(model_outputs)
# ensemble_model = Model(inputs=model_input,outputs=ensemble_outputs,name='ensemble')

In [ ]:
# ensemble_model = model.compile(optimizer='adam',loss=tf.keras.lossed.SparseCategoricalCrossentropy(from_logits=False),
#                                metrics=['accuracy'])

In [ ]:
# history = ensemble_model.fit(
#     train_ds,
#     validation_data = test_ds,
#     epochs=5
# )

In [ ]:
# import matplotlib.pyplot as plt

# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# loss = hitory.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs,acc,'r',label='Train Acc')
# plt.plot(epochs,acc,'b',label='Val Acc')
# plt.title("Training & Validation Accuracy")
# plt.legend()
# plt.figure()

# plt.plot(epochs,loss,'r',label='Train Loss')
# plt.plot(epochs,acc,'b',label='Val Loss')
# plt.title("Training & Validation Loss")
# plt.legend()

# plt.show()